This notebook performs classification on the Japanese Vowel dataset, similar to Notebook 5. However, instead of using reservoir.py's native Ridge Readout, we use scikit learn's Ridge linear regression model. We have created a new node called RidgeRegression which is compatible with all the nodes in reservoir.py. Thus, we can easily create models following the link operations. The only caveat while using scikit-learn's RidgeRegression or LinearRegression node is to have the input data and output labels as 2D matrices. This is because unlike reservoir.py's readout nodes, scikit-learn nodes do not have an option for performing partial fit. Their Linear Regression estimators fit the data at one go. Thus if you want to fit data which is in the format of [num_sequences, time series, features], you will first have to convert the data into a format such that it is [num_sequences, time series * features]. Similarly, target labels also need to be either 1-d or 2d array. 

In [1]:
import pdb
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.insert(0, '../')
from reservoirpy.datasets import japanese_vowels
from reservoirpy import set_seed, verbosity
from reservoirpy.observables import nrmse, rsquare
from reservoirpy.nodes import Reservoir, RidgeRegression, Input, ElasticNet, Lasso
from sklearn.metrics import accuracy_score

set_seed(42)
verbosity(0)

0

In [2]:
X_train, Y_train, X_test, Y_test = japanese_vowels()

In [7]:
source = Input()
reservoir = Reservoir(500, sr=0.9, lr=0.1)
readout = ElasticNet(alpha=0.001)
model = source >> reservoir >> readout

In [8]:
states_train = []
for x in X_train:
    states = reservoir.run(x, reset=True)
    states_train.append(states[-1, np.newaxis])

# the ridge regression class expects inputs and outputs as a numpy 2d matrix
# thus we do a bit of pre-processing where we convert the states_train list 
# and Y_train into a numpy array and remove any unecessary dimensions
readout.fit(np.array(states_train).squeeze(), np.array(Y_train).squeeze())

/Users/deep/opt/miniconda3/envs/pytorch2/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0833273442553355, tolerance: 0.0026666666666666666
  model = cd_fast.enet_coordinate_descent(
/Users/deep/opt/miniconda3/envs/pytorch2/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.059622463673296267, tolerance: 0.0026666666666666666
  model = cd_fast.enet_coordinate_descent(
/Users/deep/opt/miniconda3/envs/pytorch2/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.024065111105481485, tolerance: 0.0026666666666666666
  model = cd_fast.enet_coordinate_descent(
/Users/deep/

'ElasticNet-0': ElasticNet(f=<function elastic_net at 0x7fcfb9295430>, in=500, out=9)

In [9]:
states_test = []
for x in X_test:
    states = reservoir.run(x, reset=True)
    states_test.append(states[-1, np.newaxis])
y_pred = readout.run(np.array(states_test).squeeze())


In [10]:
Y_test = np.array(Y_test).squeeze()
Y_pred_class = np.argmax(y_pred, axis=1)
Y_test_class = np.argmax(Y_test, axis=1)
score = accuracy_score(Y_test_class, Y_pred_class)

print("Accuracy: ", f"{score * 100:.3f} %")

Accuracy:  98.108 %
